In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

%matplotlib inline
import torch
import torchvision
import numpy as np
from PIL import Image
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import ShuffleSplit
import torchvision.models.inception
from torchvision import transforms
import matplotlib.pyplot as plt

In [ ]:

# training
num_epochs = 20
batch_size = 32
num_workers = 6
lr = 0.001

# data sources
sample_submission = '../input/aptos2019-blindness-detection/sample_submission.csv'
root = '../input/aptos2019-blindness-detection/test_images/'
training_file = '../input/aptos2019-blindness-detection/train.csv'
trainroot = '../input/aptos2019-blindness-detection/train_images/'
pretrained = '../input/pretrained-pytorch-models/inception_v3_google-1a9a5a14.pth'
test_size = 0.2

# data preprocessing from imagenet
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# device checker, use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device", device)

# fixing random seed (for reproducibility)
seed = 555
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

In [ ]:

# Loading the pretrained inception v3
net = torchvision.models.inception_v3()
net.load_state_dict(torch.load(pretrained, map_location='cpu'))


net.fc = torch.nn.Linear(in_features=2048, out_features=5)
net.AuxLogits = torchvision.models.inception.InceptionAux(in_channels=768, num_classes=5)
net = net.to(device)

In [ ]:
optim = torch.optim.Adam(lr=lr, params=net.parameters())
crit = torch.nn.BCEWithLogitsLoss()

In [ ]:

transform = torchvision.transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:

class SimpleDataset():
    def __init__(self, data, root, transform):
        self.files = list(root + data['id_code'] + '.png')

        # LabelBinarizer takes numerical labels and returns a one-hot label
        binarizer = LabelBinarizer()
        self.targets = binarizer.fit_transform(data['diagnosis'].values)
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = Image.open(self.files[idx])
        x = self.transform(img)
        y = torch.tensor(self.targets[idx,:]).float()
        return x, y

In [ ]:

data = pd.read_csv(training_file)
ssplit = ShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)

train_index, test_index = next(ssplit.split(data['id_code']))

dataset = SimpleDataset(data.iloc[train_index], trainroot, transform)
validationset = SimpleDataset(data.iloc[test_index], trainroot, transform)

In [ ]:
# train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
# # Training model
# total_step = len(train_loader)
# for epoch in range(num_epochs):
#     for batch_i, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)

#         # Forward pass
#         outputs, hidden = net(data)
#         loss = crit(outputs, target)

#         # Backward and optimizer
#         optim.zero_grad()
#         loss.backward()
#         optim.step()

#         if (batch_i+1) % 100 == 0:
#             print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
#                  .format(epoch+1, num_epochs, batch_i+1, total_step, loss.item()))

# PATH = "./results/inception_V3_0611.pth"
# torch.save(net.state_dict(), PATH)

In [ ]:
PATH = "../input/inception-v3-0609/inception_V3_0611.pth"
# Evaluation
net.load_state_dict(torch.load(PATH))
net = net.to(device)
submit = pd.read_csv(sample_submission)
net.eval()

with torch.no_grad():
    for name in tqdm(submit['id_code']):
        img = Image.open(root+name+'.png')
        x = transform(img).to(device).unsqueeze(0)
        y = net(x).cpu().numpy()
        diag = int(np.argmax(y[:5]))
        submit.loc[submit['id_code']==name, 'diagnosis'] = diag

In [ ]:

submit.to_csv('submission.csv', index=False)
submit.head()
